In [2]:
import sys
sys.path.append('/global/homes/k/krach/usr/python_prefix_3.7/lib/python3.7/site-packages')
sys.path.append('/global/u1/k/krach/usr/ForSE')
from forse.tools.nn_tools import *
from forse.tools.img_tools import *
from forse.tools.mix_tools import *
from keras.models import Sequential, Model, load_model
from keras.layers import UpSampling2D, Conv2D, Activation, BatchNormalization
from keras.layers import Reshape, Dense, Input
from keras.layers import LeakyReLU, Dropout, Flatten, ZeroPadding2D
from keras.optimizers import Adam, RMSprop
import keras.backend as K
import os
import matplotlib.pyplot as plt
import datetime
from  functools import partial

Using TensorFlow backend.


In [3]:
class DCGAN:
    def __init__(self, output_directory, img_size):
        self.img_size = img_size
        self.channels = 1
        self.kernel_size = 5
        self.output_directory = output_directory
        self.batch_size = 32
        
    def smooth_accuracy(self, y_true, y_pred):
        return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

    def build_generator(self):
        img_shape = (self.img_size[0], self.img_size[1], self.channels)
        model = Sequential()
        model.add(Conv2D(64, kernel_size=self.kernel_size, padding="same")) # 64x64x64
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2D(128, kernel_size=self.kernel_size, padding="same", strides=2)) #32x32x128
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2D(256, kernel_size=self.kernel_size, padding="same", strides=2)) #16x16x256
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.5))
        model.add(UpSampling2D())  #32x32x128
        model.add(Conv2D(128, kernel_size=self.kernel_size, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.5))
        model.add(UpSampling2D())  #64x64x64
        model.add(Conv2D(64, kernel_size=self.kernel_size, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2D(self.channels, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("tanh"))
        img_in = Input(shape=img_shape)
        img_out = model(img_in)
        return Model(img_in, img_out)

    def build_discriminator(self):
        img_shape = (self.img_size[0], self.img_size[1], self.channels)
        model = Sequential()
        model.add(Conv2D(64, kernel_size=self.kernel_size, strides=1, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2D(128, kernel_size=self.kernel_size, strides=2, padding="same"))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2D(256, kernel_size=self.kernel_size, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        img = Input(shape=img_shape)
        validity = model(img)
        return Model(img, validity)

    def build_gan(self):
        img_shape = (self.img_size[0], self.img_size[1], self.channels)
        optimizer = Adam(0.0002, 0.9)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
        z = Input(shape=img_shape)
        img = self.generator(z)
        self.discriminator.trainable = False
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        valid = self.discriminator(img)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_gan2(self):
        img_shape = (self.img_size[0], self.img_size[1], self.channels)
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
        z = Input(shape=img_shape)
        img = self.generator(z)
        self.discriminator.trainable = False
        valid = self.discriminator(img)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        
    def build_gan3(self):
        img_shape = (self.img_size[0], self.img_size[1], self.channels)
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
        z = Input(shape=img_shape)
        img = self.generator(z)
        self.discriminator.trainable = False
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        valid = self.discriminator(img)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.discriminator = self.build_discriminator()
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])

    def train(self, epochs, patches_file, batch_size=32, save_interval=100, swap=None, seed=4324):
        self.build_gan3()
        print('GENERATOR')
        self.generator.summary()
        print('COMBINED')
        self.combined.summary()
        print('DISCRIMINATOR')
        self.discriminator.summary()
        X_train, X_test, Y_train, Y_test = load_training_set(patches_file, seed=seed)
        print("Training Data Shape: ", X_train.shape)
        half_batch = batch_size // 2
        accs = []
        for epoch in range(epochs):
            print(epoch, datetime.datetime.now().time())
            ind_batch = np.random.randint(0, X_train.shape[0], batch_size)
            g_loss = self.combined.train_on_batch(X_train[ind_batch], np.ones((batch_size, 1)))
            target_real = np.ones((half_batch, 1))
            target_fake = np.zeros((half_batch, 1))
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = Y_train[idx]
            gen_imgs = self.generator.predict(X_train[idx])
            if swap:
                swap_real = np.random.randint(0, 100, swap)
                swap_fake = np.random.randint(0, 100, swap)
                for i in range(swap):
                    if swap_real[i] < half_batch:
                        target_real[swap_real[i]] = 0
                    if swap_fake[i] < half_batch:
                        target_fake[swap_fake[i]] = 1
            d_loss_real = self.discriminator.train_on_batch(imgs, target_real)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, target_fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            acc = [d_loss_real[1], d_loss_fake[1]]
            accs.append(acc)
            # Print progress

            # If at save interval => save generated image samples, save model files
            if epoch % (save_interval) == 0:
                print(epoch)
                d_loss_real = self.discriminator.train_on_batch(imgs, target_real)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, target_fake)
                gen_imgs_test = self.generator.predict(X_test)
                val_fake = self.discriminator.evaluate(gen_imgs_test, np.zeros(len(gen_imgs_test)))
                val_real = self.discriminator.evaluate(Y_test, np.ones(len(gen_imgs_test)))
                print(val_fake, val_real)
                #print(f"{epoch} [D loss: {d_loss[0]} | D Accuracy: {100 * d_loss[1]}]")
                save_path = self.output_directory + "/models"
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                accs_to_save = np.array(accs)
                self.discriminator.save(save_path + '/discrim_'+str(epoch)+'.h5')
                self.generator.save(save_path + '/generat_'+str(epoch)+'.h5')
                np.save(save_path + '/acc_dreal_dfake_'+str(epoch)+'.npy', accs_to_save)
        self.discriminator.save(save_path + '/discrim_'+str(epoch)+'.h5')
        self.generator.save(save_path + '/generat_'+str(epoch)+'.h5')
        np.save(save_path + '/acc_dreal_dfake_'+str(epoch)+'.npy', accs_to_save)
 
    def train3(self, epochs, patches_file, batch_size=32, save_interval=100, swap=None, seed=4324):
        X_train, X_test, Y_train, Y_test = load_training_set(patches_file, seed=seed)
        print("Training Data Shape: ", X_train.shape)
        half_batch = batch_size // 2
        accs = []
        img_shape = (self.img_size[0], self.img_size[1], self.channels)
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        print('DISCRIMINATOR1')
        self.discriminator.summary()
        self.discriminator = self.build_discriminator()
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
        print('GENERATOR')
        self.generator.summary()
        z = Input(shape=img_shape)
        img = self.generator(z)
        self.discriminator.trainable = False
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        valid = self.discriminator(img)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.discriminator = self.build_discriminator()
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])
        print('COMBINED')
        self.combined.summary()
        print('DISCRIMINATOR')
        self.discriminator.summary()
        for epoch in range(epochs):
            print(epoch, datetime.datetime.now().time())
            ind_batch = np.random.randint(0, X_train.shape[0], batch_size)
            g_loss = self.combined.train_on_batch(X_train[ind_batch], np.ones((batch_size, 1)))
            target_real = np.ones((half_batch, 1))
            target_fake = np.zeros((half_batch, 1))
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = Y_train[idx]
            gen_imgs = self.generator.predict(X_train[idx])
            if swap:
                swap_real = np.random.randint(0, 100, swap)
                swap_fake = np.random.randint(0, 100, swap)
                for i in range(swap):
                    if swap_real[i] < half_batch:
                        target_real[swap_real[i]] = 0
                    if swap_fake[i] < half_batch:
                        target_fake[swap_fake[i]] = 1
            d_loss_real = self.discriminator.train_on_batch(imgs, target_real)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, target_fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            acc = [d_loss_real[1], d_loss_fake[1]]
            accs.append(acc)
            # Print progress

            # If at save interval => save generated image samples, save model files
            if epoch % (save_interval) == 0:
                print(epoch)
                d_loss_real = self.discriminator.train_on_batch(imgs, target_real)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, target_fake)
                gen_imgs_test = self.generator.predict(X_test)
                val_fake = self.discriminator.evaluate(gen_imgs_test, np.zeros(len(gen_imgs_test)))
                val_real = self.discriminator.evaluate(Y_test, np.ones(len(gen_imgs_test)))
                print(val_fake, val_real)
                #print(f"{epoch} [D loss: {d_loss[0]} | D Accuracy: {100 * d_loss[1]}]")
                save_path = self.output_directory + "/models"
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                accs_to_save = np.array(accs)
                self.discriminator.save(save_path + '/discrim_'+str(epoch)+'.h5')
                self.generator.save(save_path + '/generat_'+str(epoch)+'.h5')
                np.save(save_path + '/acc_dreal_dfake_'+str(epoch)+'.npy', accs_to_save)
        self.discriminator.save(save_path + '/discrim_'+str(epoch)+'.h5')
        self.generator.save(save_path + '/generat_'+str(epoch)+'.h5')
        np.save(save_path + '/acc_dreal_dfake_'+str(epoch)+'.npy', accs_to_save)



In [4]:
dcgan = DCGAN(output_directory='/global/homes/k/krach/scratch/NNforFG/DCGAN/tests', img_size=(64, 64))

In [5]:
training_path = '/global/homes/k/krach/scratch/NNforFG/training_set/'
training_file = 'training_set_1000patches_20x20deg_T_HR1deg_LR5deg_Npix64_set2.npy'
patch_file = training_path+training_file

In [6]:
dcgan.train(epochs=5, patches_file=patch_file, save_interval=1)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
GENERATOR
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 64, 64, 1)         2054401   
Total params: 2,054,401
Trainable params: 2,053,121
Non-trainable params: 1,280
_________________________________________________________________
COMBINED
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
model_2 (Model)              (No